## 1.  Scrape your local weather forecast.

In [111]:
import requests
from bs4 import BeautifulSoup

page = requests.get('http://web.mta.info/nyct/facts/ridership/ridership_sub.htm')
soup = BeautifulSoup(page.content, 'html.parser')
soup

<!DOCTYPE html>

<html class="no-js">
<head>
<!-- Meta -->
<meta content="width=device-width" name="viewport">
<link href="http://purl.org/dc/elements/1.1/" rel="schema.DC"/><title>National Weather Service</title><meta content="National Weather Service" name="DC.title"/><meta content="NOAA National Weather Service National Weather Service" name="DC.description"/><meta content="US Department of Commerce, NOAA, National Weather Service" name="DC.creator"/><meta content="" name="DC.date.created" scheme="ISO8601"/><meta content="EN-US" name="DC.language" scheme="DCTERMS.RFC1766"/><meta content="weather, National Weather Service" name="DC.keywords"/><meta content="NOAA's National Weather Service" name="DC.publisher"/><meta content="National Weather Service" name="DC.contributor"/><meta content="http://www.weather.gov/disclaimer.php" name="DC.rights"/><meta content="General" name="rating"/><meta content="index,follow" name="robots"/>
<!-- Icons -->
<link href="./images/favicon.ico" rel="shor

In [112]:
seven_day = soup.find(id="seven-day-forecast")
print(seven_day.prettify()) # prints with indented headers

<div class="panel panel-default" id="seven-day-forecast">
 <div class="panel-heading">
  <b>
   Extended Forecast for
  </b>
  <h2 class="panel-title">
   2 Miles ENE Chicago IL
  </h2>
 </div>
 <div class="panel-body" id="seven-day-forecast-body">
  <div id="seven-day-forecast-container">
   <ul class="list-unstyled" id="seven-day-forecast-list">
    <li class="forecast-tombstone">
     <div class="tombstone-container">
      <p class="period-name">
       Tonight
       <br>
        <br/>
       </br>
      </p>
      <p>
       <img alt="Tonight: Partly cloudy, with a low around 23. Northwest wind around 5 mph. " class="forecast-icon" src="newimages/medium/nsct.png" title="Tonight: Partly cloudy, with a low around 23. Northwest wind around 5 mph. "/>
      </p>
      <p class="short-desc">
       Partly Cloudy
      </p>
      <p class="temp temp-low">
       Low: 23 °F
      </p>
     </div>
    </li>
    <li class="forecast-tombstone">
     <div class="tombstone-container">
      

## 2. What is today's weather like?

In [113]:
# CREATE SOUP OF EXTENDED DAY FORECAST, TAKE FIRST ONE - TODAY
forecast_list = seven_day.find_all(class_='tombstone-container') #makes a list, extracts all tombstone-container divs
today = forecast_list[0]
print(today.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br>
   <br/>
  </br>
 </p>
 <p>
  <img alt="Tonight: Partly cloudy, with a low around 23. Northwest wind around 5 mph. " class="forecast-icon" src="newimages/medium/nsct.png" title="Tonight: Partly cloudy, with a low around 23. Northwest wind around 5 mph. "/>
 </p>
 <p class="short-desc">
  Partly Cloudy
 </p>
 <p class="temp temp-low">
  Low: 23 °F
 </p>
</div>


In [115]:
period = today.find(class_='period-name').get_text()
short_desc = today.find(class_='short-desc').get_text()
temp = today.find(class_='temp').get_text()

print(period)
print(short_desc)
print(temp)

Tonight
Partly Cloudy
Low: 23 °F


## 3. Extract the longer weather description for each forecast.

In [116]:
img = today.find('img')
long_desc = img['title']
long_desc

'Tonight: Partly cloudy, with a low around 23. Northwest wind around 5 mph. '

## 4. Extract the numeric temperature for all the forecasts.

In [117]:
import re
re.findall("[\d]+", temp) #regex expression, any amount of numbers

['23']

In [158]:
#numeric_temperatures = [t.get_text() for t in seven_day.select('.tombstone-container .temp')]
temp_list = seven_day.find_all(class_='temp')
temp_list = [str(item.string) for item in temp_list]
numeric_list = [str(re.findall("\d+", item)) for item in temp_list]
numeric_list = [int(item[2:4]) for item in numeric_list]
numeric_list


[23, 36, 28, 40, 37, 44, 30, 40, 32]

## 5. Store the Day, Short Forecast, Numeric Temperature, High or Low, and extended forecast in a data structure of your choice. Export your final data to a pickle object.

In [162]:
period_names = [pn.get_text() for pn in seven_day.select('.tombstone-container .period-name')]
short_descs = [sd.get_text() for sd in seven_day.select('.tombstone-container .short-desc')]
long_descs = [ld['title'] for ld in seven_day.select('.tombstone-container img')]
temps = numeric_list

weather = {}

for i in range(len(period_names)):
    weather[period_names[i]] = {}
    weather[period_names[i]]['Short Forecast'] = short_descs[i]
    weather[period_names[i]]['Long Forecast'] = long_descs[i]
    weather[period_names[i]]['Temperature'] = temps[i]
    
weather

{'M.L.KingDay': {'Long Forecast': 'M.L.King Day: A chance of snow, freezing rain, and sleet before 10am, then a chance of rain or freezing rain between 10am and 11am, then rain likely after 11am.  Cloudy, with a high near 40. Southeast wind 5 to 10 mph.  Chance of precipitation is 70%. New ice accumulation of less than a 0.1 of an inch possible.  Little or no snow and sleet accumulation expected. ',
  'Short Forecast': 'Chance WintryMix then RainLikely',
  'Temperature': 40},
 'MondayNight': {'Long Forecast': 'Monday Night: Rain.  Low around 37. Southeast wind 10 to 15 mph.  Chance of precipitation is 90%.',
  'Short Forecast': 'Rain',
  'Temperature': 37},
 'Sunday': {'Long Forecast': 'Sunday: Partly sunny, with a high near 36. Northwest wind around 5 mph becoming calm. ',
  'Short Forecast': 'Partly Sunny',
  'Temperature': 36},
 'SundayNight': {'Long Forecast': 'Sunday Night: A slight chance of snow after 1am, mixing with sleet after 5am.  Mostly cloudy, with a low around 28. South 

In [164]:
import pickle

pickle.dump(weather,open('weather.p','wb'))


In [165]:
new_weather = pickle.load(open('weather.p', 'rb'))
new_weather

{'M.L.KingDay': {'Long Forecast': 'M.L.King Day: A chance of snow, freezing rain, and sleet before 10am, then a chance of rain or freezing rain between 10am and 11am, then rain likely after 11am.  Cloudy, with a high near 40. Southeast wind 5 to 10 mph.  Chance of precipitation is 70%. New ice accumulation of less than a 0.1 of an inch possible.  Little or no snow and sleet accumulation expected. ',
  'Short Forecast': 'Chance WintryMix then RainLikely',
  'Temperature': 40},
 'MondayNight': {'Long Forecast': 'Monday Night: Rain.  Low around 37. Southeast wind 10 to 15 mph.  Chance of precipitation is 90%.',
  'Short Forecast': 'Rain',
  'Temperature': 37},
 'Sunday': {'Long Forecast': 'Sunday: Partly sunny, with a high near 36. Northwest wind around 5 mph becoming calm. ',
  'Short Forecast': 'Partly Sunny',
  'Temperature': 36},
 'SundayNight': {'Long Forecast': 'Sunday Night: A slight chance of snow after 1am, mixing with sleet after 5am.  Mostly cloudy, with a low around 28. South 